# TensorFlow Datasets

TFDS provides a collection of ready-to-use datasets for use with TensorFlow, Jax, and other Machine Learning frameworks.

It handles downloading and preparing the data deterministically and constructing a `tf.data.Dataset` (or `np.array`).

Note: Do not confuse [TFDS](https://www.tensorflow.org/datasets) (this library) with `tf.data` (TensorFlow API to build efficient data pipelines). TFDS is a high level wrapper around `tf.data`. If you're not familiar with this API, we encourage you to read [the official tf.data guide](https://www.tensorflow.org/guide/data) first.


Copyright 2018 The TensorFlow Datasets Authors, Licensed under the Apache License, Version 2.0

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/datasets/overview"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/datasets/blob/master/docs/overview.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/datasets/blob/master/docs/overview.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/datasets/docs/overview.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

## Installation

TFDS는 두 가지 패키지로 존재합니다.

- `pip install tensorflow-datasets`: The stable version, released every few months.
- `pip install tfds-nightly`: Released every day, contains the last versions of the datasets.

This colab uses `tfds-nightly`:


In [ ]:
!pip install -q tfds-nightly tensorflow matplotlib

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds

## 사용 가능한 데이터세트 찾기

모든 데이터세트 빌더는 `tfds.core.DatasetBuilder`의 서브 클래스입니다. 사용 가능한 빌더의 목록을 얻으려면, `tfds.list_builders()`를 사용하거나 [카탈로그](https://www.tensorflow.org/datasets/catalog/overview)를 살펴보세요.

In [ ]:
tfds.list_builders()

## 데이터세트 로드하기

### tfds.load

데이터세트를 로드하는 가장 쉬운 방법은 `tfds.load`입니다.

1. 데이터를 다운로드하여 [`tfrecord`](https://www.tensorflow.org/tutorials/load_data/tfrecord) 파일로 저장합니다.
2. `tfrecord`를 로드하고 `tf.data.Dataset`를 생성합니다.


In [ ]:
ds = tfds.load('mnist', split='train', shuffle_files=True)
assert isinstance(ds, tf.data.Dataset)
print(ds)

몇 가지 일반적인 인수:

- `split=`: 읽을 분할(예: `'train'`, `['train', 'test']`, `'train[80%:]'`,...). [분할 API 가이드](https://www.tensorflow.org/datasets/splits)를 참조하세요.
- `shuffle_files=`: 각 epoch 간에 파일을 셔플할지 여부를 제어합니다(TFDS는 큰 데이터세트를 여러 개의 작은 파일에 저장합니다).
- `data_dir=`: 데이터세트가 저장된 위치(기본값은 `~/tensorflow_datasets/`)
- `with_info=True`: 데이터세트 메타 데이터를 포함하는 `tfds.core.DatasetInfo`를 반환합니다.
- `download=False`: 다운로드를 비활성화합니다.


### tfds.builder

`tfds.load`는 `tfds.core.DatasetBuilder`를 둘러싼 얇은 래퍼입니다. `tfds.core.DatasetBuilder` API를 사용하여 같은 출력을 얻을 수 있습니다.

In [ ]:
builder = tfds.builder('mnist')
# 1. Create the tfrecord files (no-op if already exists)
builder.download_and_prepare()
# 2. Load the `tf.data.Dataset`
ds = builder.as_dataset(split='train', shuffle_files=True)
print(ds)

### `tfds build` CLI

If you want to generate a specific dataset, you can use the [`tfds` command line](https://www.tensorflow.org/datasets/cli). For example:

```sh
tfds build mnist
```

See [the doc](https://www.tensorflow.org/datasets/cli) for available flags.

## 데이터세트 반복하기

### dict

기본적으로 `tf.data.Dataset` 객체에는 `tf.Tensor`의 `dict`가 포함됩니다.

In [ ]:
ds = tfds.load('mnist', split='train')
ds = ds.take(1)  # Only take a single example

for example in ds:  # example is `{'image': tf.Tensor, 'label': tf.Tensor}`
  print(list(example.keys()))
  image = example["image"]
  label = example["label"]
  print(image.shape, label)

To find out the `dict` key names and structure, look at the dataset documentation in [our catalog](https://www.tensorflow.org/datasets/catalog/overview#all_datasets). For example: [mnist documentation](https://www.tensorflow.org/datasets/catalog/mnist).

### As tuple (`as_supervised=True`)

`as_supervised=True`를 사용하면 감독된 데이터세트 대신 튜플 `(features, label)`을 얻을 수 있습니다.

In [ ]:
ds = tfds.load('mnist', split='train', as_supervised=True)
ds = ds.take(1)

for image, label in ds:  # example is (image, label)
  print(image.shape, label)

### As numpy (`tfds.as_numpy`)

`tfds.as_numpy`를 사용하여 변환합니다.

- `tf.Tensor` -&gt; `np.array`
- `tf.data.Dataset` -&gt; `Iterator[Tree[np.array]]` (`Tree` can be arbitrary nested `Dict`, `Tuple`)


In [ ]:
ds = tfds.load('mnist', split='train', as_supervised=True)
ds = ds.take(1)

for image, label in tfds.as_numpy(ds):
  print(type(image), type(label), label)

### As batched tf.Tensor (`batch_size=-1`)

`batch_size=-1`을 사용하여 전체 데이터세트를 단일 배치로 로드할 수 있습니다.

This can be combined with `as_supervised=True` and `tfds.as_numpy` to get the the data as `(np.array, np.array)`:


In [ ]:
image, label = tfds.as_numpy(tfds.load(
    'mnist',
    split='test',
    batch_size=-1,
    as_supervised=True,
))

print(type(image), image.shape)

Be careful that your dataset can fit in memory, and that all examples have the same shape.

## Benchmark your datasets

Benchmarking a dataset is a simple `tfds.benchmark` call on any iterable (e.g. `tf.data.Dataset`, `tfds.as_numpy`,...).


In [ ]:
ds = tfds.load('mnist', split='train')
ds = ds.batch(32).prefetch(1)

tfds.benchmark(ds, batch_size=32)
tfds.benchmark(ds, batch_size=32)  # Second epoch much faster due to auto-caching

* Do not forget to normalize the results per batch size with the `batch_size=` kwarg.
* In the summary, the first warmup batch is separated from the other ones to capture `tf.data.Dataset` extra setup time (e.g. buffers initialization,...).
* Notice how the second iteration is much faster due to [TFDS auto-caching](https://www.tensorflow.org/datasets/performances#auto-caching).
* `tfds.benchmark` returns a `tfds.core.BenchmarkResult` which can be inspected for further analysis.

### 엔드 투 엔드 파이프라인 빌드하기

더 진행하려면, 다음을 살펴볼 수 있습니다.

- 전체 훈련 파이프라인(배치 처리, 셔플링 등)을 확인하는 [엔드 투 엔드 Keras 예제](https://www.tensorflow.org/datasets/keras_example)
- Our [performance guide](https://www.tensorflow.org/datasets/performances) to improve the speed of your pipelines (tip: use `tfds.benchmark(ds)` to benchmark your datasets).


## Visualization

### tfds.as_dataframe

`tf.data.Dataset` objects can be converted to [`pandas.DataFrame`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html) with `tfds.as_dataframe` to be visualized on [Colab](https://colab.research.google.com).

* Add the `tfds.core.DatasetInfo` as second argument of `tfds.as_dataframe` to visualize images, audio, texts, videos,...
* Use `ds.take(x)` to only display the first `x` examples. `pandas.DataFrame` will load the full dataset in-memory, and can be very expensive to display.

In [ ]:
ds, info = tfds.load('mnist', split='train', with_info=True)

tfds.as_dataframe(ds.take(4), info)

### tfds.show_examples

`tfds.show_examples` returns a `matplotlib.figure.Figure` (only image datasets supported now):

In [ ]:
ds, info = tfds.load('mnist', split='train', with_info=True)

fig = tfds.show_examples(ds, info)

## 데이터세트 메타 데이터에 액세스하기

모든 빌더에는 데이터세트 메타 데이터를 포함하는 `tfds.core.DatasetInfo` 객체가 포함됩니다.

다음을 통해 액세스할 수 있습니다.

- `tfds.load` API:


In [ ]:
ds, info = tfds.load('mnist', with_info=True)

- `tfds.core.DatasetBuilder` API:

In [ ]:
builder = tfds.builder('mnist')
info = builder.info

데이터세트 정보에는 데이터세트에 대한 추가 정보(버전, 인용, 홈페이지, 설명 등)가 포함됩니다.

In [ ]:
print(info)

### 특성 메타 데이터(레이블 이름, 이미지 형상 등)

`tfds.features.FeatureDict`에 액세스합니다.

In [ ]:
info.features

클래스 수, 레이블 이름:

In [ ]:
print(info.features["label"].num_classes)
print(info.features["label"].names)
print(info.features["label"].int2str(7))  # Human readable version (8 -> 'cat')
print(info.features["label"].str2int('7'))

형상, dtype:

In [ ]:
print(info.features.shape)
print(info.features.dtype)
print(info.features['image'].shape)
print(info.features['image'].dtype)

### 분할 메타 데이터(예: 분할 이름, 예제 수 등)

`tfds.core.SplitDict`에 액세스합니다.

In [ ]:
print(info.splits)

사용 가능한 분할:

In [ ]:
print(list(info.splits.keys()))

Get info on individual split:

In [ ]:
print(info.splits['train'].num_examples)
print(info.splits['train'].filenames)
print(info.splits['train'].num_shards)

하위 분할 API와도 동작합니다.

In [ ]:
print(info.splits['train[15%:75%]'].num_examples)
print(info.splits['train[15%:75%]'].file_instructions)

## Troubleshooting

### Manual download (if download fails)

If download fails for some reason (e.g. offline,...). You can always manually download the data yourself and place it in the `manual_dir` (defaults to `~/tensorflow_datasets/download/manual/`.

To find out which urls to download, look into:

 * For new datasets (implemented as folder): [`tensorflow_datasets/`](https://github.com/tensorflow/datasets/tree/master/tensorflow_datasets/)`<type>/<dataset_name>/checksums.tsv`. For example: [`tensorflow_datasets/text/bool_q/checksums.tsv`](https://github.com/tensorflow/datasets/blob/master/tensorflow_datasets/text/bool_q/checksums.tsv).

   You can find the dataset source location in [our catalog](https://www.tensorflow.org/datasets/catalog/overview).
 * For old datasets: [`tensorflow_datasets/url_checksums/<dataset_name>.txt`](https://github.com/tensorflow/datasets/tree/master/tensorflow_datasets/url_checksums)

### Fixing `NonMatchingChecksumError`

TFDS ensure determinism by validating the checksums of downloaded urls.
If `NonMatchingChecksumError` is raised, might indicate:

  * The website may be down (e.g. `503 status code`). Please check the url.
  * For Google Drive URLs, try again later as Drive sometimes rejects downloads when too many people access the same URL. See [bug](https://github.com/tensorflow/datasets/issues/1482)
  * The original datasets files may have been updated. In this case the TFDS dataset builder should be updated. Please open a new Github issue or PR:
     * Register the new checksums with `tfds build --register_checksums`
     * Eventually update the dataset generation code.
     * Update the dataset `VERSION`
     * Update the dataset `RELEASE_NOTES`: What caused the checksums to change ? Did some examples changed ?
     * Make sure the dataset can still be built.
     * Send us a PR

Note: You can also inspect the downloaded file in `~/tensorflow_datasets/download/`.


## 인용

If you're using `tensorflow-datasets` for a paper, please include the following citation, in addition to any citation specific to the used datasets (which can be found in the [dataset catalog](https://www.tensorflow.org/datasets/catalog/overview)).

```
@misc{TFDS,
  title = { {TensorFlow Datasets}, A collection of ready-to-use datasets},
  howpublished = {\url{https://www.tensorflow.org/datasets}},
}
```